In [7]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

In [9]:
#change device to cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuda


In [11]:
#class same as that of trained model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(256 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 5)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.leaky_relu(self.conv1(x), negative_slope=0.01))
        x = self.pool(F.leaky_relu(self.conv2(x), negative_slope=0.01))
        x = self.pool(F.leaky_relu(self.conv3(x), negative_slope=0.01))
        x = self.pool(F.leaky_relu(self.conv4(x), negative_slope=0.01))
        x = x.view(-1, 256 * 16 * 16)
        x = self.dropout(F.leaky_relu(self.fc1(x), negative_slope=0.01))
        x = self.fc2(x)
        return x

In [13]:
model = SimpleCNN().to(device)

In [15]:
# Instantiate the model and load the state dictionary
model = SimpleCNN()
model.load_state_dict(torch.load('simple_cnn_model.pth'))
model.eval()

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=5, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [36]:
#transform function for model
def transform_image():
    return transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

In [38]:
# Load and preprocess the image
def preprocess_image(image_path):
    transform = transform_image()
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image

In [40]:
#function to predict image class
def predict_image_class(model, image, classes):
    
    #shift image and model to cuda if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = image.to(device)
    model = model.to(device)
    
    # Predict the class
    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)
    
    return classes[predicted_class.item()]


In [46]:
if __name__ == "__main__":
    # Define the classes of the model
    classes = ['Bar', 'Casino', 'Hospital', 'Library', 'Restaurant']
    
    # Path to the image
    image_path = "W:/Concordia/Summer2024/AI/test/images/random/download1.jpeg"
    
    #preprocess image
    image = preprocess_image(image_path)
    
    # Predict the image class
    predicted_class = predict_image_class(model, image, classes)
    print(f'Predicted Class: {predicted_class}')

Predicted Class: Bar
